# 89-2-integer-production-resources
Testing the new solve

In [1]:
import itertools
import numpy as np
from collections import OrderedDict as OD

from gpkit import Model, Variable

from gpx.manufacturing import QNACell, FabLine, ConwipTooling
from gpx.primitives import Process

In [2]:
# create some processesc
process_times = [
    ('first', 15),
    ('second', 20),
    ('third', 25),
]

processes_times = OD()
for pt in process_times:
    processes_times[pt[0]] = pt[1]

cv = Variable('CV', 0.5, '-', 'coeffcient of variation')

# create processes
processes = {name: Process(cv=cv) for name,time in processes_times.items()}

# make cells from processes
cells = OD()
for name in processes_times.keys():
    cells[name] = QNACell(processes[name])
# cells = {name: QNACell(proc) for name,proc in processes.items()}

# make the fab line
line = FabLine(list(cells.values()))

# create constraints
constr = [line]

# make the costs
costs = np.sum([c.m for c in cells.values()]) + line.L

# create the model
m = Model(costs, constr)

called GPX model  23
called GPX model  23
called GPX model  23
called GPX model  23


In [3]:
import gpx

In [4]:
gm = gpx.Model(costs, [line])
gm.cost = costs

called GPX model  23


In [5]:
# update the substitutions
## process times
## process times
gm.substitutions.update({proc.t : processes_times[name] for name,proc in processes.items()})

## rate
gm.substitutions[line.lam] = 7

## CV
gm.substitutions[cv] = 0.75

In [6]:
print(gm.solve().table())

Using solver 'cvxopt'
 for 32 free variables
  in 34 posynomial inequalities.
Solving took 0.38 seconds.

             ┃┓             ┓           ┓
             ┃┃             ┃           ┃
             ┃┃             ┃           ┣╸QNACell2.W╶⎨
             ┃┃             ┃           ┛ (33.1min)
             ┃┃             ┃           ┓
             ┃┣╸L           ┣╸FabLine.W ┣╸QNACell1.W╶⎨
             ┃┃ (9.46count) ┃ (1.35hr)  ┛ (27.1min)
             ┃┃             ┃           ┓
             ┃┃             ┃           ┣╸QNACell.W╶⎨
        Cost╺┫┛             ┛           ┛ (20.9min)
 (19.7count) ┃┓             ┓
             ┃┃             ┃
             ┃┣╸QNACell2.m  ┣╸Process3.t
             ┃┛ (4.14count) ┛ (25min, fixed)
             ┃┓             ┓
             ┃┣╸QNACell1.m  ┣╸Process2.t
             ┃┛ (3.42count) ┛ (20min, fixed)
             ┃┓             ┓
             ┃┣╸QNACell.m   ┣╸Process1.t
             ┃┛ (2.68count) ┛ (15min, fixed)



       ┃┓         ┓
     

In [7]:
# find the list of the varkeys that will be discrete
discvars = [cell.m for cell in cells.values()]
discvars.append(line.L)

In [8]:
min_rate = 6.8

In [9]:
_ = gm.SolveDiscrete(
    discrete_resources=discvars,
    target_variable=line.lam,
    target_value=min_rate,
    new_cost=line.lam**-1
)

Using solver 'cvxopt'
 for 29 free variables
  in 34 posynomial inequalities.
Solving took 0.251 seconds.
Using solver 'cvxopt'
 for 29 free variables
  in 34 posynomial inequalities.
Solving took 0.278 seconds.
Using solver 'cvxopt'
 for 28 free variables
  in 34 posynomial inequalities.
Solving took 0.176 seconds.


In [10]:
gm.solution['discretevariables']

{QNACell.m: 3.0, QNACell1.m: 3.0, QNACell2.m: 4.0, FabLine.L: 10.0}

In [11]:
gm.solution['cost']

20.000000004993293

In [12]:
print(gm.solution.table())


                  ┃┓                  ┓           ┓
                  ┃┃                  ┃           ┃
                  ┃┃                  ┃           ┣╸QNACell2.W
                  ┃┃                  ┃           ┛ (34.8min)
                  ┃┃                  ┃           ┓
                  ┃┣╸L                ┣╸FabLine.W ┃
                  ┃┃ (10count, fixed) ┃ (1.47hr)  ┣╸QNACell1.W
                  ┃┃                  ┃           ┛ (32.7min)
                  ┃┃                  ┃           ┣╸QNACell.W
             Cost╺┫┛                  ┛           ┛ (20.7min)
 (20count, fixed) ┃┓
                  ┃┃
                  ┃┣╸QNACell2.m
                  ┃┛ (4count, fixed)
                  ┃┓
                  ┃┣╸QNACell.m
                  ┃┛ (3count, fixed)
                  ┃┓
                  ┃┣╸QNACell1.m
                  ┃┛ (3count, fixed)



       ┃┓
 Model╺┫┃
       ┃┣╸m = 3count
       ┃┛


Free Variables
--------------
         | FabLine
       W : 1.471   [hr

In [13]:
print(gm.solution['discretesol'].table())


                 ┃┓
            Cost╺┫┃
 (0.143hr/count) ┃┣╸FabLine.\lambda^-1
                 ┃┛ (0.143hr/count)



       ┃┓
       ┃┃
       ┃┃
       ┃┣╸QNACell1╶⎨
       ┃┃
       ┃┛
       ┃┓
       ┃┃
       ┃┣╸FabLine╶⎨
 Model╺┫┃
       ┃┛
       ┃┓
       ┃┃
       ┃┣╸QNACell2╶⎨
       ┃┛
       ┃┣╸QNACell╶⎨
       ┃┛
       ┃┣╸Process2╶⎨
       ┃┣╸Process3╶⎨
       ┃┣╸CV = 0.75


Free Variables
--------------
         | FabLine
       W : 1.435   [hr]       Total flow time
 \lambda : 6.97    [count/hr] Line Continuous Production Rate

         | QNACell
       W : 18.9    [min]      Total flow time through cell
      Wq : 3.897   [min]      Expected queueing time
  \alpha : 0.4192             (1-rho)
 \lambda : 6.97    [count/hr] Production rate
     c2a : 0.5625             Arrival coefficient of variation squared
     c2d : 0.5625             Departure coefficient of variation squared
     c2s : 0.5625             Process coefficient of variation squared
     rho : 0.5808